In [57]:
#destination source length

def parse_map_block(lines):
    ret_val = []
    for l in lines:
        sp = tuple(int(x) for x in l.split())
        ret_val.append((sp[0] - sp[1], sp[1], sp[1] + sp[2] - 1 ))
        
    #By sorting the output we can guarantee that the starting points of each individual mapping
    #within a map will be increasing, allowing many extra checks later to be skipped
    return sorted(ret_val, key=lambda x: x[1])

def parse_in():
    lines = open("Inputs/Day05.txt").read().strip().split('\n')
    seeds = [int(x) for x in lines[0].split()[1:]]
    
    blanks = [i for i,l in enumerate(lines) if l == ""]
    blanks.append(len(lines))
    
    maps = []
    for b in zip(blanks,blanks[1:]):
        maps.append(parse_map_block(lines[b[0]+2:b[1]]))

    return seeds, maps

def parse_test():
    text = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""
    lines = text.strip().split('\n')
    seeds = [int(x) for x in lines[0].split()[1:]]
    
    blanks = [i for i,l in enumerate(lines) if l == ""]
    blanks.append(len(lines))
    
    maps = []
    for b in zip(blanks,blanks[1:]):
        maps.append(parse_map_block(lines[b[0]+2:b[1]]))

    return seeds, maps
seeds, maps = parse_in()

In [59]:
def translate(seed, map_):
    for m in map_:
        if seed >= m[1] and seed <= m[2]:
            return m[0] + seed
    return seed

def part1(seeds, almanac):
    ret_val = 100000000000000000
    #6 translations to do
    for seed in seeds:
        s = seed
        for a in almanac:
            s = translate(s, a)
        ret_val = min(ret_val, s)
    return ret_val
print(part1(seeds, maps)) #should be 309796150
%timeit part1(seeds, maps)

309796150
76.4 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
def translate_range(range_in, map_):
    new_ranges = []
    
    while range_in:
        start, end = range_in.pop(0)
        #determine overlap with each potential mapping
        for m0, m1, m2 in map_:
            if start <= m2 and end >= m1:
                #this overlaps at some point
                overlap = (max(start, m1), min(end, m2))
                #translate this overlap
                new_ranges.append((m0 + overlap[0], m0 + overlap[1]))
                
                #there is some leftover at the front but the maps in the almanac were sorted by their starting location
                #meaning no future map will be affecting this range.  Move it to the new_ranges immediately
                if overlap[0] > start:
                    new_ranges.append((start, overlap[0] - 1))
                #There is some leftover at the end which an upcoming range might touch
                if overlap[1] < end:
                    range_in.append((overlap[1] + 1, end))
                break
        else:
            new_ranges.append((start, end))

    return new_ranges
                
        

def part2(seeds, almanac):
    ranges = [(x[0], x[0] + x[1] - 1) for x in list(zip(seeds,seeds[1:]))[::2]]

    for m in almanac:
        ranges = translate_range(ranges, m)

    return min((r[0] for r in ranges))
    

print(part2(seeds, maps)) # should be 50716416
%timeit part2(*parse_in())

50716416
533 µs ± 3.65 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
print("Read in Data")
%timeit parse_in()
print("Part 1 w/o read data")
%timeit part1(seeds, maps)
print("Part 2 w/o read data")
%timeit part2(seeds, maps)
print("Part 1 w/o read data")
%timeit part1(*parse_in())
print("Part 2 w/o read data")
%timeit part2(*parse_in())

Read in Data
200 µs ± 3.66 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Part 1 w/o read data
76 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Part 2 w/o read data
315 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Part 1 w/o read data
285 µs ± 6.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Part 2 w/o read data
548 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
